### 1) Power Analysis for means

In [77]:
import numpy as np 
import scipy.stats as st
from scipy.stats import ttest_ind, mannwhitneyu
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.meta_analysis import effectsize_smd 

def effect_size_mean(n_total, alpha, power, alternative, m1, m2, sd):

    N = int(n_total/2)
    eff = abs(effectsize_smd(mean2=m2, sd2=sd, nobs2=N, mean1=m1, sd1=sd, nobs1=N)[0])
    n = int(tt_ind_solve_power(effect_size=eff, alpha=alpha, power=power, ratio=1.0, alternative=alternative)) 
    print("The total sample size needed to have %.f%% power is %.d." % (100*power, 2*n))
    print("The effect size is %.3f" % (eff))
    return n, eff

def power_simulation_mean(n, alpha, alternative, m1, eff, sd, k=20000):
    p_values = np.zeros(k)  
    p_valuesWMW = np.zeros(k)  
    for i in range(k):
        x1 = np.random.normal(m1, sd, n) 
        x2 = np.random.normal(eff, sd, n) 
        _, p_values[i] = st.ttest_ind(x1, x2, equal_var=True, alternative=alternative)
        _, p_valuesWMW[i] = st.mannwhitneyu(x1, x2, alternative=alternative)
    power_t_test = np.mean(p_values < alpha) 
    print(f"Power of T test: %.3f%%" % (100*power_t_test))
    power_WMW_test = np.mean(p_valuesWMW < alpha)
    print(f"Power of Mann-Whitney test: %.3f%%" % (100*power_WMW_test))

In [79]:
alpha = 0.05 
power = 0.80
alternative = 'two-sided' 

m1 = 0
m2 = 0.5 
sd = 1
n_total = 2000 

n, eff = effect_size_mean(n_total, alpha, power, alternative, m1, m2, sd)
power_simulation_mean(n, alpha, alternative, m1, eff, sd)

The total sample size needed to have 80% power is 126.
The effect size is 0.500
Power of T test: 79.600%
Power of Mann-Whitney test: 77.775%


### 2) Power Analysis for proportions 

In [39]:
from statsmodels.stats.power import zt_ind_solve_power
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.proportion import proportions_chisquare
from statsmodels.stats.proportion import proportion_effectsize

def power_simulation_proportion(n, alpha, alternative, p1, p2, k=20000):
    p_values = np.zeros(k)  
    p_values_chi = np.zeros(k)  
    for i in range(k):
        control_sample = st.binom.rvs(1, p1, size=n)
        test_sample = st.binom.rvs(1, p2, size=n) 
        _, p_values[i] = proportions_ztest(count=[sum(control_sample), sum(test_sample)], 
                                           nobs=[n, n], alternative=alternative)
        _, p_values_chi[i], _ = proportions_chisquare(count=[sum(control_sample), sum(test_sample)], 
                                                      nobs=[n, n])
    power_z_test = np.mean(p_values < alpha) 
    print(f"Power of Z test: %.3f%%" % (100*power_z_test))
    power_chi_test = np.mean(p_values_chi < alpha)
    print(f"Power of Chi-square test: %.3f%%" % (100*power_chi_test))
    
def effect_size_proportion(alpha, power, alternative, p1, p2):
    eff = abs(proportion_effectsize(prop1=p1, prop2=p2, method='normal'))
    n = int(zt_ind_solve_power(effect_size=eff, alpha=alpha, power=power, ratio=1.0, alternative=alternative)) 
    print("The total sample size needed to have %.f%% power is %.d." % (100*power, 2*n))
    print("The effect size is %.3f" % (eff))
    return n, eff

In [81]:
alpha = 0.05 
power = 0.80
alternative = 'two-sided' 

p1 = 0.4
p2 = 0.5

n, eff = effect_size_proportion(alpha, power, alternative, p1, p2)
power_simulation_proportion(n, alpha, alternative, p1, p2)

The total sample size needed to have 80% power is 774.
The effect size is 0.201
Power of Z test: 79.390%
Power of Chi-square test: 79.460%


### 3) Power Analysis for means - non normal distribution 

In [83]:
def power_simulation_mean_non_normal(alpha, alternative, m1, eff, sd, k=20000):
    a = 1.99
    p_values = np.zeros(k)  
    p_valuesWMW = np.zeros(k)  
    for i in range(k):
        x1 = st.gamma.rvs(a, loc=m1, scale=sd, size=n)
        x2 = st.gamma.rvs(a, loc=eff, scale=sd, size=n) 
        _, p_values[i] = st.ttest_ind(x1, x2, equal_var=True, alternative=alternative)
        _, p_valuesWMW[i] = st.mannwhitneyu(x1, x2, alternative=alternative)
    power_t_test = np.mean(p_values < alpha) 
    print(f"Power of T test: %.3f%%" % (100*power_t_test))
    power_WMW_test = np.mean(p_valuesWMW < alpha)
    print(f"Power of Mann-Whitney test: %.3f%%" % (100*power_WMW_test))

In [85]:
alpha = 0.05 
power = 0.80
alternative = 'two-sided' 

m1 = 0
m2 = 0.5 
sd = 1
n_total = 2000 

n, eff = effect_size_mean(n_total, alpha, power, alternative, m1, m2, sd)
power_simulation_mean_non_normal(alpha, alternative, m1, eff, sd)

The total sample size needed to have 80% power is 126.
The effect size is 0.500
Power of T test: 51.705%
Power of Mann-Whitney test: 66.565%
